### run the code below to make graph object and its methods available 


In [37]:
import json
import os
import lib_info as lib_info
import matplotlib.pyplot as plt
import networkx as nx
def dfs(vertex : str, visited : list, v_neighbors : dict , components : dict, root: str):
    visited.append(vertex)
    components[root].append(vertex)
    for neighbor, weight in v_neighbors[vertex]:
        if neighbor not in visited:
            dfs(neighbor, visited, v_neighbors,components,root)
class Graph: 
    def __init__(self):
        pass 
        self.vertices=set()
        self.v_neighbors={}
        self.vertices_data={}
        self.components={}
        self.visited=[]
        self.in_edges={}
        self.out_vertices=set()

    def add_vertex(self, vertex,v_data):

        if vertex not in self.vertices:
            self.vertices.add(vertex)
            self.v_neighbors[vertex]=[]
            self.vertices_data[vertex]=v_data
            self.in_edges[vertex]=0

    def add_edge(self, vertex1, vertex2, weight=1):
        if vertex1 in self.vertices and vertex2 in self.vertices:
            self.v_neighbors[vertex2].append((vertex1, weight))
            self.in_edges[vertex1]=self.in_edges[vertex1]+1
        else : 
            self.out_vertices.add(vertex1)

    def find_components(self):
        self.visited=[]

        for vertex in self.vertices:
            if self.in_edges[vertex]==0:
                root=vertex
                self.components[vertex]=[]
                dfs(vertex, self.visited, self.v_neighbors, self.components, root)

    def get_nighbors(self, vertex_id:str , number_of_neighbors:int):
        elements=[]
        for i in range(number_of_neighbors):
            try :
                elements.append(self.ver[vertex_id]["id"])
            except :
                pass
        neighbors=[]
        pass 

    def count_moments(self):
        pass

def get_json_graph(file_path):

    try : 
        with open(file_path, 'r' , encoding="utf-8") as file:
            return  json.load(file)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")

def create_graph(json_data:dict):

    graph=Graph()

    for message_id in json_data:
        graph.add_vertex(message_id,json_data[message_id])

    for message_id, message in json_data.items():

        if not message["replyTo"]==None:
            replyer_id=str(message["replyTo"])
            try : 

                weight=json_data[message_id]["userId"]

            except Exception as e : 

                weight=None

            graph.add_edge(message_id, replyer_id,weight=weight)
    return graph

base_path = lib_info.get_peoject_path()
file_path=os.path.join(base_path, "messages_json_labeled.json")
json_data= get_json_graph(file_path)
graph=create_graph(json_data)
graph.find_components()

### examples below will show you how to use the graph object

In [65]:
message_info=graph.vertices_data.get("66080",{}) # to get info about message with id 66080

In [39]:
date_time=message_info.get("datetime","") # to get datetime of message with id 66080

In [44]:
# example getting text of message with id  :
id="66080"
text=graph.vertices_data.get(id,{}).get("message","")
print("message text is : " ,text)

message text is :  I have 10 clients, requests keep coming in, my own projects — it’s impossible to keep everything in my head, I need to respond within 24 hours.


In [40]:
import datetime
# convert date_time to seconds, minutes, hours
date_time_second=datetime.datetime.fromisoformat(date_time).strftime('%S')
date_time_min=datetime.datetime.fromisoformat(date_time).strftime('%M')
date_time_hour=datetime.datetime.fromisoformat(date_time).strftime('%H')

In [41]:
print(f"Time : {date_time_hour}:{date_time_min}:{date_time_second}") # print time of message with id 66080  

Time : 10:20:17


In [42]:
time_sec=int(date_time_second) + int(date_time_min)*60 + int(date_time_hour)*60*60 
print(f"Time in seconds : {time_sec}")

Time in seconds : 37217


In [87]:
def message_time_difference_seconds(message_id1, message_id2) -> int: 
    """
    this function takes two message IDs and returns the difference in seconds between their timestamps
    """

    message_info1 = graph.vertices_data.get(str(message_id1), {})  #fetching message data
    message_info2 = graph.vertices_data.get(str(message_id2), {})  

    date_time1 = message_info1.get("datetime", "") #get time
    date_time2 = message_info2.get("datetime", "")

    if not date_time1 or not date_time2: #check if timestamp exists
        print("Error: Missing timestamp info")
        return -1  

    dt1 = datetime.datetime.fromisoformat(date_time1) #convert to datetime object
    dt2 = datetime.datetime.fromisoformat(date_time2)

    time_difference = abs((dt1 - dt2).total_seconds()) #calculate time difference

    return int(time_difference) #and voila



In [92]:
message_time_difference_seconds(66080, 66081)

7

In [98]:
def split_message_by_symbol(symbol: str, message_id: str) -> list:
    """
    example : 
    message id : 12345 -> message text : "hello,world,this,is,a,test." 
    symbol : ','
    then the function will return -> ["hello", "world", "this", "is", "a", "test."]
    """
    message_text = graph.vertices_data.get(str(message_id), {}).get("message", "Message not found")    
    split_message = message_text.split(symbol)
    
    return split_message

In [99]:
def is_it_question(message_id: str) -> bool:
    """
    this function will get id of a message and return True if the message is a question, False otherwise
    """
    
    question_words = ["what", "how", "why", "where", "who", "when", "is", "are", "can", "could", "would", "do", "did"]

    message_text = graph.vertices_data.get(str(message_id), {}).get("message", "Message not found").lower().strip() 

    if message_text.endswith('?') or any(message_text.startswith(word) for word in question_words): #checking if function ends with a questionmark or starts with a question word
        return True #im not sure whether it should be "and" instead of "or", because then its more exclusive
    
    return False


In [103]:
is_it_question(66061)

True

In [110]:
def message_user_id(message_id: str) -> str:
    """
    this function will get id of a message and return the user id who sent the message
    """
    user_id = graph.vertices_data.get(str(message_id), {}).get("userId", "User not found")
    return user_id


In [112]:
message_user_id(66070)

'user44095265'

In [125]:
def messages_by_users(*message_ids: list) -> dict:
    """
    input : list of message ids ["12345", "67890", "34567"] 
    this function will get a list of message ids and return a dictionary where the keys are the user ids and the values are the list of message ids sent by the user
    output example : {"userid12323": ["12345", "67890"], "userid2323": ["34567"]} 
    """
    user_messages = {}

    for message_id in message_ids: #azzurro! i do da iteration! 
        user_id = message_user_id(message_id)
        user_messages.setdefault(user_id, set()).add(message_id) #mamma mia give a pasta bowl (set) to every user

    for user_id in user_messages: #i converta da seta to da lista mamma mia
        user_messages[user_id] = list(user_messages[user_id])

    return user_messages #bellissimo! 

In [130]:
messages_by_users(66080, 66090, 66100, 66333)

{'user172556084': [66080],
 'user142752811': [66090],
 'user44095265': [66100],
 'user105512464': [66333]}